# **스타벅스 정보 크롤링**

In [37]:
import time
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

In [43]:
def fetch_starbucks():
    starbucks_url = 'https://www.starbucks.co.kr/index.do'
    driver = webdriver.Chrome()
    driver.maximize_window()
    driver.get(starbucks_url)
    time.sleep(1)

    # ACtionChains(): 마우스나 키보으와 같은 복잡한 사용자 상호작용을 시뮬레이션하는 데 사용
    action = ActionChains(driver)
    first_tag = driver.find_element(By.CSS_SELECTOR, '#gnb > div > nav > div > ul > li.gnb_nav03')
    # #gnb > div > nav > div > ul > li.gnb_nav03 > div > div > div > ul:nth-child(1) > li:nth-child(3) > a
    second_tag = driver.find_element(By.CSS_SELECTOR, '#gnb > div > nav > div > ul > li.gnb_nav03 > div > div > div > ul:nth-child(1) > li:nth-child(3) > a')
    action.move_to_element(first_tag).move_to_element(second_tag).click().perform() # perform(): ActionChains()를 실행
    time.sleep(1)
    seoul_tag = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, '#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(1) > a'))
    )
    seoul_tag.click()
    time.sleep(3)

    # 매장 정보를 담기 위한 빈 리스트
    store_list = []
    addr_list = []
    lat_list = []
    lng_list = []

    # set_gugun_cd_btn
    # #mCSB_2_container > ul > li:nth-child(1) > a
    WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'set_gugun_cd_btn')))
    gu_elements= driver.find_elements(By.CLASS_NAME, 'set_gugun_cd_btn')
    gu_elements[0].click()
    time.sleep(5)

    # quickSearchResultBoxSidoGugun
    WebDriverWait(driver, 5).until(EC.presence_of_all_elements_located((By.CLASS_NAME, 'quickSearchResultBoxSidoGugun')))

    req = driver.page_source
    soup = BeautifulSoup(req) # parsing
    stores = soup.find('ul', 'quickSearchResultBoxSidoGugun').find_all('li')

    for store in stores:
        store_name = store.find('strong').text
        store_addr = store.find('p').text
        store_addr = re.sub(r'\d{4}-\d{4}$', '', store_addr).strip()
        store_lat = store['data-lat']
        store_lng = store['data-long']
        store_list.append(store_name)
        addr_list.append(store_addr)
        lat_list.append(store_lat)
        lng_list.append(store_lng)
    
    df = pd.DataFrame({
        'store': store_list,
        'addr': addr_list,
        'lat': lat_list,
        'lng': lng_list
    })

    driver.quit()
    return df

In [44]:
starbucks_df = fetch_starbucks()
starbucks_df

,store,addr,lat,lng
0,역삼아레나빌딩,서울특별시 강남구 언주로 425 (역삼동),37.501087,127.043069
1,논현역사거리,서울특별시 강남구 강남대로 538 (논현동),37.510178,127.022223
2,신사역성일빌딩,서울특별시 강남구 강남대로 584 (논현동),37.5139309,127.0206057
3,국기원사거리,서울특별시 강남구 테헤란로 125 (역삼동),37.499517,127.031495
4,대치재경빌딩,서울특별시 강남구 남부순환로 2947 (대치동),37.494668,127.062583
...,...,...,...,...
625,상봉역,서울특별시 중랑구 망우로 307 (상봉동),37.59689,127.08647
626,묵동,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층",37.615368,127.076633
627,망우동,서울특별시 중랑구 망우로 407 (망우동),37.5995083771241,127.097538751682
628,양원역,서울특별시 중랑구 양원역로10길 3 (망우동),37.6066536267232,127.106359790053


In [40]:
starbucks_df.to_csv('starbucks_seoul.csv', index=False, encoding='utf-8-sig')
print('데이터가 starbucks_seoul.csv 파일로 저장되었습니다.')

데이터가 starbucks_seoul.csv 파일로 저장되었습니다.
